# The prologue

Let me start with a situation from my life. When I reached an intermediate level of my datascience journey, I faced with this issue of huge datasets. The datasets are becoming bigger and with my laptop having average specs (I5--7th gen with 12 Gb of Ram), it feels like training of the models takes forever. Especially the memory demanding algorithms such as cross validation is taking heavy toll on my lap and me (as i have to run the algorithm over night and this makes me wakeup several times in the middle of night to check for execution completion).  

So to tackle this situation I found several methods 

*) AWS --- Costly but effective --- allows you to assign huge resources for your algorithms

*) paid Kernels ---- Same as Aws 

*) ........................

*) Last but not the least, we have spark --- free open source 

So this notebook is my Entire Solution for handling huge datasets in data science, in other words a solution for better sleep at night. 


## Preknowledge requirement


* Spark --- Begginer (This notebook is for absolute beginners of spark. For those who use spark already can still take some knowledge as we all think differently)

* Datascience --- Beginner, intermediate , advanced ( only thing is you should be new/intermediate to spark )

* Python --- Should love it.

# Table of Contents

#### 1). Introduction.

#### 2). Importing and checking Datasets.

     2.1) Creating a DataFrame.
     
     2.2) Showing the data.
     
     2.3) Grabbing the data.
     
     2.4) Creating new columns.

#### 3). Basics operations on dataframes.

     3.1) Filtering data using SQL Queries.
  
     3.2) Filtering data using Dataframe methods.

#### 4). Handling missing data using spark.

     4.1) Keeping the missing data.
  
     4.2) Dropping the missing data.
  
     4.3) Filling the missing data.

#### 5). How spark works internally.

#### 6). Conclusion and further materials.

#### 7). Resources


Feel free to skip to sections as per your requirements.

# 1)  Introduction to pyspark

This is a begginer friendly notebook for people who want to dive into the world of big data and datascience. As the datasets becomes more and more bigger, the need for bigdata technologies such as hadoop, spark etc.. increases.In my research I found spark to be the best one among those technologies. This notebook is written for absolute begginers in spark. This notebook will be accompanied by several notebooks explaining more advanced topics in pyspark.
     

Apache Spark is a distributed framework that handle's Big Data analysis. Apache Spark is written in Scala and can be integrated with Python, Scala, R, SQL, Java  languages. Spark is basically a computational engine, that works with huge sets of data by processing them in parallel and batch systems.PySpark is the Python API of Apache Spark.I like to keep the introduction short and want to show how to work using pyspark. For curious minds i'll be explaining how spark works internally in section 7.


The main advantage of spark is that, it can do various machine learning tasks with ease.The Ml lib library which we see in the upcoming notebooks will help you to deal with datascience problems using huge datasets. 


# 2) Importing and Checking datasets.

Post Spark version2.0, Spark introduced dataframes.These are like advanced version of tables with rows and columns,for easy handling of large datasets.

These dataframes are the same ones which you might have used in python or R with additional properties. Spark DataFrame expand on a lot of these concepts, allowing you to transfer that knowledge easily by understanding the simple syntax of Spark DataFrames.

### 2.1) Creating a DataFrame

First we need to start a SparkSession:

The below 2 commands are used to import findspark module which actually tells the program where spark folder (the one we downloads during installation) is residing in your computer.


In [ ]:
#import findspark

In [ ]:
#findspark.init('/home/davinci/spark-2.4.5-bin-hadoop2.7')

In [2]:
!pip install pyspark  #for installing spark in kaggle kernel
from pyspark.sql import SparkSession

     |████████████████████████████████| 217.8 MB 4.3 kB/s  eta 0:00:01
     |████████████████████████████████| 197 kB 51.1 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218258791 sha256=39bf2010f97be87e187ce978be6c3d74d97521058922c1158deeb084f5378acd
  Stored in directory: /root/.cache/pip/wheels/01/c0/03/1c241c9c482b647d4d99412a98a5c7f87472728ad41ae55e1e
Successfully built pyspark


Then start the SparkSession

In [3]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()

We will first need to get the data from a file (or connect to a large distributed file like HDFS, we'll talk about this later once we move to larger datasets on AWS EC2). Here i'm importing data from a json file

In [4]:
#This is a dataset available online 
# Might be a little slow locally
df = spark.read.json('../input/peoplejson1/people.json')

### 2.2) Showing the data

In [5]:
# Note how data is missing!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Spark will automatically set null value for missing data.

In [6]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



when we ran the above command spark understood the schema of the dataframe. This is an handy tool while using some datasets.

There are data types that makes infering of  schema easier (like csv ). 

Still there is an option to define our own schema and can read the datasets in that schema.

Spark has all the tools you need for this, it just requires a very specific structure:

In [7]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

We have to create the list of Structure fields
    * :param name: string, name of the field.
    * :param dataType: :class:`DataType` of the field.
    * :param nullable: boolean, whether the field can be null (None) or not.

In [8]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]
final_struc = StructType(fields=data_schema)
df = spark.read.json('../input/peoplejson1/people.json', schema=final_struc)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In the above piece of code we defined a a list of data schema using the parmaeters. Then by using the structtype we made a schema out of it. Then in 3rd sentense, we used that schema to read our json file. 

To show the columns

In [9]:
df.columns

['age', 'name']

To show the rows

In [10]:
df.head(2) #by default shows 1 row

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

### 2.3) Grabbing the data

In [11]:
print(df['age'])

Column<b'age'>


As you can see this returns a column object. But it is far less usefull. we can use .select() function to import it as a dataframe.

In [12]:
print(df.select('age'))

DataFrame[age: int]


In [13]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



There are 2 types of functions .show() and .collect()

*) .show()  : print the data and shows to us

*) .collect() : can be used to assign data to a variable.

To select more than one column, we can give the column names as a list

In [14]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### 2.4) Creating new columns

To create new columns we can use .withcolumn()

In [15]:
# Adding a new column which is copied from an old column.
df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [16]:
# Renaming a column
df.withColumnRenamed('age','supernewage').show()

+-----------+-------+
|supernewage|   name|
+-----------+-------+
|       null|Michael|
|         30|   Andy|
|         19| Justin|
+-----------+-------+



More complicated operations to create new columns

In [17]:
df.withColumn('doubleage',df['age']*4).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|      120|
|  19| Justin|       76|
+----+-------+---------+



# 3) Basic operations on Dataframes

### 3.1) Filtering Data using SQL

A big ability of spark is quick filtering and searching capacity. Spark also allows us to use sql queries for filtering and other operations. So i'll be showing both sql and ordinary DF methods.

In [18]:
# Let Spark know about the header and infer the Schema types!
df = spark.read.csv('../input/applstock/appl_stock.csv',inferSchema=True,header=True)

In [19]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [20]:
# Using SQL
df.filter("Close<500").show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [21]:
# Using SQL with .select()
df.filter("Close<500").select(['Open','Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



### 3.2) Filtering data using normal df methods.

In [22]:
# Using normal df methods.
df.filter(df["Close"] < 200).show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-22 00:00:00|206.78000600000001|        207.499996|            197.16|            197.75|220441900|         25.620401|
|2010-01-28 00:00:00|        204.930004|        205.500004|        198.699995|        199.289995|293375600|25.819922000000002|
|2010-01-29 00:00:00|        201.079996|        202.199995|        190.250002|        192.060003|311488100|         24.883208|
|2010-02-01 00:00:00|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02 00:00:00|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.3

In [23]:
# Make sure to add in the parenthesis separating the statements!
df.filter( (df["Close"] < 200) & (df['Open'] > 200) ).show()

+-------------------+------------------+----------+----------+----------+---------+------------------+
|               Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+-------------------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28 00:00:00|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29 00:00:00|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+-------------------+------------------+----------+----------+----------+---------+------------------+



In [24]:
# Make sure to add in the parenthesis separating the statements!
df.filter( (df["Close"] < 200) | (df['Open'] > 200) ).show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [25]:
# Make sure to add in the parenthesis separating the statements!
df.filter( (df["Close"] < 200) & ~(df['Open'] < 200) ).show()

+-------------------+------------------+----------+----------+----------+---------+------------------+
|               Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+-------------------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28 00:00:00|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29 00:00:00|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+-------------------+------------------+----------+----------+----------+---------+------------------+



In [26]:
df.filter(df["Low"] == 197.16).show()

+-------------------+------------------+----------+------+------+---------+---------+
|               Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+-------------------+------------------+----------+------+------+---------+---------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+-------------------+------------------+----------+------+------+---------+---------+



# 4) Handling Missing Data

There will be missing data or null data in your datasets. SO you can handle these missing data in three ways

    *) Keep the missing data points.
    *) Drop them missing data points.
    *) Fill them in with some other value.



### 4.1) Keeping the missing data

Here we dont have to do anything. Spark already understands the missing value and gives it null.

In [27]:
df = spark.read.csv("../input/containsnull1/ContainsNull.csv",header=True,inferSchema=True)

In [28]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Notice how the data remains as a null.


### 4.2) Drop the missing data

We can yse .na functions for missing data.

There are 3 parameters.

    df.na.drop(how='any', thresh=None, subset=None)
    
    * param how: 'any' or 'all'.
    
        If 'any', drop a row if it contains any nulls.
        If 'all', drop a row only if all its values are null.
    
    * param thresh: int, default None
    
        If specified, drop rows that have less than `thresh` non-null values.
        This overwrites the `how` parameter.
        
    * param subset: 
        optional list of column names to consider.

In [29]:
# Drop any row that contains missing data
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [30]:
# Has to have at least 2 NON-null values
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [31]:
df.na.drop(subset=["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [32]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [33]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### 4.3) Fill the missing values

We can also fill the missing values with new values. If you have multiple nulls across multiple data types, Spark is actually smart enough to match up the data types. For example:

In [34]:
df.na.fill('NEW VALUE').show()

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| null|
|emp2|NEW VALUE| null|
|emp3|NEW VALUE|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



In [35]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Usually you should specify what columns you want to fill with the subset parameter

In [36]:
df.na.fill('No Name',subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



A very common practice is to fill values with the mean value for the column, for example:

In [37]:
from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Sales'])).show()



+----------+
|avg(Sales)|
+----------+
|     400.5|
+----------+



# 5) How spark works internally.



You might have thought like why this session is kept here instead of placing it in the beginning. The sole purpose of it is that, this section explains how spark works internally and to understand we should know some terms such as sparkcontest and all which you will be quite familiar by now.



The whole code you have written for execution is called as driver program. Spark runs this programs on various clusters known as nodes. This Parallelization is the one helping you to achieve greater processing speeds. Spark uses executors for this. If you are using shells and writting the programs in an interactive way, these shells are considered as driver programs by itself. The next thing is sc -- the spark contest (we have used it in our code). This is actually the linkage between the driver program and the multiple executors. So whenever we reads a file the spark contest internally links to all the nodes. 



This is a basic introduction of how spark works internally.


# 6) Conclusion

As you have seen spark is a good way to work with huge datasets.Spark helps to solve the problem of larger processing times by using parallelization approach. 

The spark topic is huge.I wont be able to show everthing to you in one notebook. My sole purpose of creating this notebook was to introduce you to Pyspark and the basic operations on datasets using pyspark. I hope that is fulfilled. 

In the upcoming notebooks I'll be diving into the spark ML lib library used for machine learning and will introduce you to various machine learning algorithms using pyspark.

1). Linear Regression.


2). Logistic Regression.


3). Decision Trees & Random forests.


4). K-means Clustering.


5). Natural Language Processing.



and more...



# 7) Resources

First of all a big thanks to Jose Portilla and his course "Spark and Python for big data using pyspark" in udemy, for lifting of my yspark journe. Great course with lots of valuable materials and examples. 

https://spark.apache.org/ : The spark website -- Literally have everything we need 


A lot of youtube videos and websites.




Hope to meet you soon in next notebook.